### 시상작업용 스크립트(비공개시상)
1. 코드
2. 시상파일을 열어서 파일,시트,추출컬럼을 지정 후 작업(n번은 작업순번 유의)
3. n번 작업결과를 join

In [1]:
import pandas as pd
import duckdb
from openpyxl import load_workbook

path='d:/사용인코드(KJ).xlsx'
cd_df = pd.read_excel(path,usecols='C,E,G',dtype={'사번':object})
cd_df.columns = ['사번','이름','지점']

In [5]:
path ='E:/글로벌백업(180329)/04.수수료작업/202405시상/시상파일/비공개/'
path2 = '20240710_정산_손보_1차_배포(총괄)/'
file_nm = '20240710_AF_01_202212_KB손보_영업활성화(총괄)_배포.xlsx'
ym = '202212' #시상업적월
r_path = f'{path}{path2}{file_nm}'
# data = pd.read_excel(f'{path}',sheet_name=None)
# data.keys()
# wb = load_workbook(r_path,read_only=True,data_only=True)
# wb.sheetnames

In [18]:
file_nm = '20240710_AF_03_202301_KB손보_영업활성화(재정산)_배포.xlsx'
r_path = f'{path}{path2}{file_nm}'
헤더 =  4
'''작업순서를 지정하고 1번~3번까지 작업가능함..2번이상 작업한 경우 아래의 join query로 합산처리'''

seq = 2 #작업순서 ; 시상, 2:
sht = 'KB_영업활성화(23.01)'
총괄, 시상, 사번 = 'bm','bl','h'
환수 = ''  #None
환수 = None if 환수=='' else 환수
''' ------------------------------ '''
if seq > 1:
    df_3rd = pd.DataFrame()

hdr = 헤더-1 
dic = {'총괄':총괄,'사번':사번,'시상':시상}
if 환수:
    dic['환수'] = 환수
dic = dict(sorted(dic.items(),key=lambda item: str(len(item[1]))+item[1]))
use_cols= list(dic.values())

df = pd.read_excel(f'{r_path}',header=hdr,sheet_name=sht,usecols=','.join(use_cols),
                  dtype=str)
df.columns=dic.keys()
print('read count: ', len(df))
df = df[df['총괄']=='GLAD']
df = df.astype({'시상':float})
if 환수:
    df = df.astype({'환수':float})
if "*" in df.iloc[0,0]:
    df['사번'] = df['사번'].str.replace('*','')

# join
c_name = 'a.환수'
query_string = f'''
     SELECT c.지점, c.이름, sum(a.시상) as 시상, c.사번 as 사번,
            sum({ 0 if 환수 is None else c_name}) as 환수
       FROM df a left outer JOIN cd_df c 
         ON a.사번 = c.사번
       GROUP BY c.지점, c.이름, c.사번 '''
df_result = duckdb.query(query_string).to_df()
print('error :', df_result[df_result['사번']==None])

if seq==1:
    df_1st = df_result.copy()
elif seq==2:
    df_2nd = df_result.copy()
elif seq==3:
    df_3rd = df_result.copy()

df_result.insert(2,"업적월",ym)
df_result['확정'] = df_result['시상']
df_result['시상'] = 0
df_result.drop(columns='사번').to_clipboard(index=False, header=False)
print("please paste clipboard to excel")
print(df_result.head(2), '\n')
print('Total 시상:', df_result['확정'].sum() , ' 환수:', df_result['환수'].sum() if 환수 else 0)

read count:  10689
error : Empty DataFrame
Columns: [지점, 이름, 시상, 사번, 환수]
Index: []
please paste clipboard to excel
      지점   이름     업적월  시상       사번   환수         확정
0  글래드조은  김명주  202212   0  4025050  0.0   584640.0
1  글래드조은  김민화  202212   0  4025091  0.0  1964499.0 

Total 시상: 13061326.0  환수: 0


In [ ]:
# df_2nd['시상'].sum()
# df_1st = df_2nd.copy()
df_1st['시상'] = df_1st['시상'] * 1000

In [15]:
df_result[['지점','이름','업적월','시상','환수','확정']].to_clipboard(index=False)

In [19]:
''' 최초,확정 시상을 join '''
#확정시상이 두 개일경우 합친 후 조인
df_all = pd.concat([df_2nd,df_3rd]).groupby(['지점','이름'])[['시상','환수']].sum().reset_index()

query_string = f'''
     SELECT a.지점, a.이름, a.시상 as 최초, a.환수 as 환수, b.시상 as 확정
       FROM df_1st a  full outer JOIN df_all b
         ON a.지점 = b.지점 
         AND a.이름 = b.이름
        '''
df_res = duckdb.query(query_string).to_df()
df_res.insert(2,"업적월",ym)
# df_res.insert(4,"환수",0)
df_res.to_clipboard(index=False, header=False)
print(df_res.head(2),'\n')
print('Total 최초:', df_res['최초'].sum(), ' 환수:', df_res['환수'].sum(), '확정:', df_res['확정'].sum() )

      지점   이름     업적월          최초   환수         확정
0  글래드조은  김명주  202212   365400.00  0.0   584640.0
1  글래드조은  김민화  202212  1227812.25  0.0  1964499.0 

Total 최초: 8487111.5  환수: 0.0 확정: 13061326.0


In [ ]:
print(df_all['시상'].sum())
print(df_2nd['시상'].sum())
print(df_3rd['시상'].sum())